In [149]:
from multiprocessing import Pool
import pandas as pd
import numpy as np
import allel
import pysam
import json
import os
from datetime import datetime
pd.set_option('display.max_columns', None)
from urllib import request
from tqdm import tqdm
from autocnv import settings
from gtfparse import read_gtf


In [150]:
work_dir = settings.BASE_DIR
raw_data_dir = os.path.join(work_dir, 'raw_data')
result_data_dir = os.path.join(work_dir, 'data')
if not os.path.exists(raw_data_dir):
    os.mkdir(raw_data_dir)
if not os.path.exists(result_data_dir):
    os.mkdir(result_data_dir)

# version_date = datetime.today().strftime('%Y-%m-%d')
version_date = '2021-10-28'
version_date_dir = os.path.join(raw_data_dir, version_date)
if not os.path.exists(version_date_dir):
    os.mkdir(version_date_dir)
print(
        f"raw_data_dir: {raw_data_dir}",
        f"result_data_dir: {result_data_dir}",
        f"version_date_dir: {version_date_dir}",
        sep="\n"
      )

raw_data_dir: /staging/software/lib/python3.9/site-packages/autocnv-1-py3.9.egg/raw_data
result_data_dir: /staging/software/lib/python3.9/site-packages/autocnv-1-py3.9.egg/data
version_date_dir: /staging/software/lib/python3.9/site-packages/autocnv-1-py3.9.egg/raw_data/2021-10-28


In [151]:
exon_url = 'https://hgdownload.soe.ucsc.edu/goldenPath/hg19/bigZips/genes/hg19.refGene.gtf.gz'
cyto_band_url = 'https://hgdownload.soe.ucsc.edu/goldenPath/hg19/database/cytoBand.txt.gz'
gene_info_url = 'https://ftp.ncbi.nih.gov/refseq/H_sapiens/Homo_sapiens.gene_info.gz'
ref_gene_url = 'https://hgdownload.soe.ucsc.edu/goldenPath/hg19/database/refGene.txt.gz'
clingen_gene_curation_url = 'https://ftp.clinicalgenome.org/ClinGen_gene_curation_list_GRCh37.tsv'
clingene_region_curation_url = 'https://ftp.clinicalgenome.org/ClinGen_region_curation_list_GRCh37.tsv'
clinvar_url = 'https://ftp.ncbi.nlm.nih.gov/pub/clinvar/vcf_GRCh37/clinvar.vcf.gz'
hi_pred_url = 'https://www.deciphergenomics.org/files/downloads/HI_Predictions_Version3.bed.gz'
gnomad_lof_url = 'https://datasetgnomad.blob.core.windows.net/dataset/release/2.1.1/constraint/gnomad.v2.1.1.lof_metrics.by_gene.txt.bgz'
gnomad_control_only_url = 'https://datasetgnomad.blob.core.windows.net/dataset/papers/2019-sv/gnomad_v2.1_sv.controls_only.sites.bed.gz'
hgnc_gene_fam_url = 'ftp://ftp.ebi.ac.uk/pub/databases/genenames/hgnc/csv/genefamily_db_tables/family.csv'
# ori file
a=[exon_url, cyto_band_url, gene_info_url, ref_gene_url, clingen_gene_curation_url, clingene_region_curation_url, clinvar_url, hi_pred_url, gnomad_lof_url, gnomad_control_only_url, hgnc_gene_fam_url]

In [152]:
exon_ori_file = os.path.join(version_date_dir, 'hg19.refGene.gtf.gz ')
cyto_band_ori_file = os.path.join(version_date_dir, 'cytoBand.txt.gz')
gene_info_ori_file = os.path.join(version_date_dir, 'Homo_sapiens.gene_info.gz')
ref_gene_ori_file = os.path.join(version_date_dir, 'refGene.txt.gz')
clingen_gene_ori_file = os.path.join(version_date_dir, 'clingen_gene_hg19.tsv')
clingen_region_ori_file = os.path.join(version_date_dir, 'clingen_region_hg19.tsv')
clinvar_ori_vcf_file = os.path.join(version_date_dir, 'clinvar.vcf.gz')
hi_pred_ori_file = os.path.join(version_date_dir, 'HI_Predictions_Version3.bed.gz')
gnomad_lof_ori_file = os.path.join(version_date_dir, 'gnomad.v2.1.1.lof_metrics.by_gene.txt.bgz')
gnomad_control_ori_file = os.path.join(version_date_dir, 'gnomad_v2.1_sv.controls_only.sites.bed.gz')
hgnc_gene_fam_file = os.path.join(version_date_dir, 'family.csv')
# result file
b = [exon_ori_file, cyto_band_ori_file, gene_info_ori_file, ref_gene_ori_file, clingen_gene_ori_file, clingen_region_ori_file, clinvar_ori_vcf_file, hi_pred_ori_file, gnomad_lof_ori_file, gnomad_control_ori_file, hgnc_gene_fam_file]

In [153]:
# Download data
import subprocess

for i, j in zip(b, a):
    if not os.path.exists(i):
        # print(f'downloading {i} from {j}')
        # request.urlretrieve(exon_url, exon_ori_file)
        commands = f'wget -c -P {version_date_dir} {j}'
        task = subprocess.run(
                ["wget", "-c", "-O", i, j],
                encoding='utf-8',
        )
        if task.returncode == 0:
            print(f'{i} done')
        else:
            print(f'Rerun: {commands}')

--2024-05-11 13:59:54--  https://hgdownload.soe.ucsc.edu/goldenPath/hg19/bigZips/genes/hg19.refGene.gtf.gz
Connecting to 192.168.134.12:1080... connected.
Proxy request sent, awaiting response... 200 OK
Length: 21846291 (21M) [application/x-gzip]
Saving to: ‘/staging/software/lib/python3.9/site-packages/autocnv-1-py3.9.egg/raw_data/2021-10-28/hg19.refGene.gtf.gz ’

     0K .......... .......... .......... .......... ..........  0% 27.2K 13m2s
    50K .......... .......... .......... .......... ..........  0% 37.1K 11m16s
   100K .......... .......... .......... .......... ..........  0% 32.5K 11m7s
   150K .......... .......... .......... .......... ..........  0% 45.0K 10m17s
   200K .......... .......... .......... .......... ..........  1% 78.4K 9m6s
   250K .......... .......... .......... .......... ..........  1% 76.7K 8m20s
   300K .......... .......... .......... .......... ..........  1% 50.5K 8m6s
   350K .......... .......... .......... .......... ..........  1% 52.5K 7m55s


/staging/software/lib/python3.9/site-packages/autocnv-1-py3.9.egg/raw_data/2021-10-28/hg19.refGene.gtf.gz  done


. .......... .......... .......... .......... 97%  242K 1s
 20900K .......... .......... .......... .......... .......... 98% 1.55M 1s
 20950K .......... .......... .......... .......... .......... 98% 5.85M 1s
 21000K .......... .......... .......... .......... .......... 98% 8.40M 1s
 21050K .......... .......... .......... .......... .......... 98% 9.14M 1s
 21100K .......... .......... .......... .......... .......... 99% 7.20M 0s
 21150K .......... .......... .......... .......... .......... 99% 1.38M 0s
 21200K .......... .......... .......... .......... .......... 99% 5.22M 0s
 21250K .......... .......... .......... .......... .......... 99% 19.8M 0s
 21300K .......... .......... .......... ....                 100% 36.3M=46s

2024-05-11 14:00:42 (464 KB/s) - ‘/staging/software/lib/python3.9/site-packages/autocnv-1-py3.9.egg/raw_data/2021-10-28/hg19.refGene.gtf.gz ’ saved [21846291/21846291]



In [154]:
dgv_ori_file = os.path.join(raw_data_dir, 'DGV.GS.March2016.50percent.GainLossSep.Final.hg19.gff3')

In [155]:
os.system(f'wget -c -P {raw_data_dir} http://dgv.tcag.ca/dgv/docs/DGV.GS.March2016.50percent.GainLossSep.Final.hg19.gff3')

--2024-05-11 14:00:42--  http://dgv.tcag.ca/dgv/docs/DGV.GS.March2016.50percent.GainLossSep.Final.hg19.gff3
Connecting to 192.168.134.12:1080... connected.
Proxy request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



0

In [156]:
gene_file = os.path.join(result_data_dir, 'gene.sorted.bed')
omim_gene_file = os.path.join(result_data_dir, 'omim-gene.sorted.bed')
clinvar_file = os.path.join(result_data_dir, 'clinvar-pathogenic.sorted.vcf')
decipher_gene_file = os.path.join(result_data_dir, 'decipher-gene.sorted.bed')
dgv_gain_file = os.path.join(result_data_dir, 'dgv-gain.sorted.bed')
dgv_loss_file = os.path.join(result_data_dir, 'dgv-loss.sorted.bed')
func_region_file = os.path.join(result_data_dir, 'func-region.sorted')
gnomad_del_file = os.path.join(result_data_dir, 'gnomad-del.sorted.bed')
gnomad_dup_file = os.path.join(result_data_dir, 'gnomad-dup.sorted.bed')
hi_cds_file = os.path.join(result_data_dir, 'hi-cds.sorted.bed')
hi_exon_file = os.path.join(result_data_dir, 'hi-exon.sorted.bed')
hi_gene_file = os.path.join(result_data_dir, 'hi-gene.sorted.bed')
hi_region_file = os.path.join(result_data_dir, 'hi-region.sorted.bed')
ts_gene_file = os.path.join(result_data_dir, 'ts-gene.sorted.bed')
ts_region_file = os.path.join(result_data_dir, 'ts-region.sorted.bed')
uhi_gene_file = os.path.join(result_data_dir, 'uhi-gene.sorted.bed')
uhi_region_file = os.path.join(result_data_dir, 'uhi-region.sorted.bed')
uts_gene_file = os.path.join(result_data_dir, 'uts-gene.sorted.bed')
uts_region_file = os.path.join(result_data_dir, 'uts-region.sorted.bed')


# Download require data

if not os.path.exists(exon_ori_file):
    print(f'downloading gene exon file')
    request.urlretrieve(exon_url, exon_ori_file)
    print('done')

if not os.path.exists(cyto_band_ori_file):
    print(f'downloading cytoband conf file to {cyto_band_ori_file}')
    request.urlretrieve(cyto_band_url, cyto_band_ori_file)
    print('done!')

if not os.path.exists(gene_info_ori_file):
    print(f'downloading gene info to {gene_info_ori_file}')
    request.urlretrieve(gene_info_url, gene_info_ori_file)
    print('done!')

if not os.path.exists(ref_gene_ori_file):
    print(f'downloading ref gene to {ref_gene_ori_file}')
    request.urlretrieve(ref_gene_url, ref_gene_ori_file)
    print('done!')

if not os.path.exists(clingen_gene_ori_file):
    print(f'downloading clingen gene list to {clingen_gene_ori_file}')
    request.urlretrieve(clingen_gene_curation_url, clingen_gene_ori_file)
    print('done!')

if not os.path.exists(clingen_region_ori_file):
    print(f'downloading clingen region file to {clingen_region_ori_file}')
    request.urlretrieve(clingene_region_curation_url, clingen_region_ori_file)
    print('done!')

if not os.path.exists(clinvar_ori_vcf_file):
    print(f'downloading clingen region file to {clinvar_ori_vcf_file}')
    request.urlretrieve(clinvar_url, clinvar_ori_vcf_file)
    print('done!')

if not os.path.exists(hi_pred_ori_file):
    print(f'downloading hi prediction file to {hi_pred_ori_file}')
    request.urlretrieve(hi_pred_url, hi_pred_ori_file)
    print('done!')
if not os.path.exists(gnomad_lof_ori_file):
    print(f'downloading pLoF file from GnomAD to {gnomad_lof_ori_file}')
    request.urlretrieve(gnomad_lof_url, gnomad_lof_ori_file)
    print('done!')
if not os.path.exists(gnomad_control_ori_file):
    print(f'downloading gnomad control only file to {gnomad_control_ori_file}')
    request.urlretrieve(gnomad_control_only_url, gnomad_control_ori_file)
    print('done!')

if not os.path.exists(hgnc_gene_fam_file):
    print(f'downloading gene family file to {hgnc_gene_fam_file}')
    request.urlretrieve(hgnc_gene_fam_url, hgnc_gene_fam_file)
    print('done!')

In [157]:
cyto_band_df = pd.read_csv(
    cyto_band_ori_file, sep='\t',
    names=['#chrom', 'start', 'end', 'name', 'gieStain']
)
cyto_band_df.to_csv(settings.CYTO_BAND_FILE.strip('.gz'), sep='\t', index=False)
print(settings.CYTO_BAND_FILE.strip('.gz'))

/staging/software/lib/python3.9/site-packages/autocnv-1-py3.9.egg/data/cyto-band.bed


In [158]:
!bgzip -cf {settings.CYTO_BAND_FILE.strip(".gz")} > {settings.CYTO_BAND_FILE}
!tabix -fp bed {settings.CYTO_BAND_FILE}
!rm {settings.CYTO_BAND_FILE.strip(".gz")}
print(settings.CYTO_BAND_FILE)

/staging/software/lib/python3.9/site-packages/autocnv-1-py3.9.egg/data/cyto-band.bed.gz


In [159]:
cols = [
    'bin', 'name', 'chrom', 'strand', 'txStart', 'txEnd', 'cdsStart', 'cdsEnd', 'exonCount', 'exonStarts', 'exonEnds',
    'score', 'name2', 'cdsStartStat', 'cdsEndStat', 'ExonFrames']
refgene = pd.read_csv(ref_gene_ori_file, sep='\t', names=cols)
print(ref_gene_ori_file)
refgene = refgene[~refgene['chrom'].str.match(r'.*fix$')]
refgene['length'] = refgene['cdsEnd'] - refgene['cdsStart']
refgene = refgene.sort_values('length', ascending=False)
refgene = refgene.drop_duplicates('name2', keep='first')
refgene.head()

/staging/software/lib/python3.9/site-packages/autocnv-1-py3.9.egg/raw_data/2021-10-28/refGene.txt.gz


,bin,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds,score,name2,cdsStartStat,cdsEndStat,ExonFrames,length
10573,26,NM_001351365,chr1,+,144146811,146467744,144146846,146466121,93,"144146811,144148789,144149726,144150981,144151...","144147021,144148892,144149941,144151054,144151...",0,NBPF19,cmpl,cmpl,"0,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,...",2319275
10459,26,NM_001278267,chr1,+,144146810,146467744,144158383,146466121,131,"144146810,144148789,144149726,144150981,144151...","144147021,144148892,144149941,144151054,144151...",0,NBPF20,cmpl,cmpl,"-1,-1,-1,-1,-1,-1,-1,-1,0,2,1,2,1,2,1,2,1,2,1,...",2307738
18617,12,NM_000109,chrX,-,31137340,33357505,31140035,33357382,79,"31137340,31144758,31152218,31164407,31165391,3...","31140047,31144790,31152311,31164531,31165635,3...",0,DMD,cmpl,cmpl,"0,1,1,0,2,2,2,2,2,0,2,0,1,2,1,1,2,1,0,0,1,0,2,...",2217347
29979,2,NM_001351274,chr11,-,83166055,85339417,83170860,85337661,27,"83166055,83173044,83177750,83180243,83182668,8...","83170967,83173136,83177860,83180416,83182770,8...",0,DLG2,cmpl,cmpl,"1,2,0,1,1,1,0,2,2,1,0,2,2,2,1,0,1,2,0,0,0,0,0,...",2166801
68033,9,NM_033225,chr8,-,2792882,4852436,2796106,4851938,70,"2792882,2799993,2806820,2807752,2808635,281174...","2796266,2800126,2806908,2807865,2808797,281179...",0,CSMD1,cmpl,cmpl,"2,1,0,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...",2055832


In [160]:
gene_info = pd.read_csv(gene_info_ori_file, sep='\t')
gene_info.head()

,#tax_id,GeneID,Symbol,LocusTag,Synonyms,dbXrefs,chromosome,map_location,description,type_of_gene,Symbol_from_nomenclature_authority,Full_name_from_nomenclature_authority,Nomenclature_status,Other_designations,Modification_date,Feature_type
0,9606,1,A1BG,-,A1B|ABG|GAB|HYST2477,MIM:138670|HGNC:HGNC:5|Ensembl:ENSG00000121410...,19,19q13.43,alpha-1-B glycoprotein,protein-coding,A1BG,alpha-1-B glycoprotein,O,alpha-1B-glycoprotein|HEL-S-163pA|epididymis s...,20240503,-
1,9606,2,A2M,-,A2MD|CPAMD5|FWP007|S863-7,MIM:103950|HGNC:HGNC:7|Ensembl:ENSG00000175899...,12,12p13.31,alpha-2-macroglobulin,protein-coding,A2M,alpha-2-macroglobulin,O,alpha-2-macroglobulin|C3 and PZP-like alpha-2-...,20240505,-
2,9606,3,A2MP1,-,A2MP,HGNC:HGNC:8|Ensembl:ENSG00000291190|AllianceGe...,12,12p13.31,alpha-2-macroglobulin pseudogene 1,pseudo,A2MP1,alpha-2-macroglobulin pseudogene 1,O,pregnancy-zone protein pseudogene,20231010,-
3,9606,9,NAT1,-,AAC1|MNAT|NAT-1|NATI,MIM:108345|HGNC:HGNC:7645|Ensembl:ENSG00000171...,8,8p22,N-acetyltransferase 1,protein-coding,NAT1,N-acetyltransferase 1,O,arylamine N-acetyltransferase 1|N-acetyltransf...,20240403,-
4,9606,10,NAT2,-,AAC2|NAT-2|PNAT,MIM:612182|HGNC:HGNC:7646|Ensembl:ENSG00000156...,8,8p22,N-acetyltransferase 2,protein-coding,NAT2,N-acetyltransferase 2,O,arylamine N-acetyltransferase 2|N-acetyltransf...,20240502,-


In [161]:
refgene_info = refgene.merge(gene_info, left_on='name2', right_on='Symbol')
refgene_info.head()

,bin,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds,score,name2,cdsStartStat,cdsEndStat,ExonFrames,length,#tax_id,GeneID,Symbol,LocusTag,Synonyms,dbXrefs,chromosome,map_location,description,type_of_gene,Symbol_from_nomenclature_authority,Full_name_from_nomenclature_authority,Nomenclature_status,Other_designations,Modification_date,Feature_type
0,26,NM_001351365,chr1,+,144146811,146467744,144146846,146466121,93,"144146811,144148789,144149726,144150981,144151...","144147021,144148892,144149941,144151054,144151...",0,NBPF19,cmpl,cmpl,"0,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,...",2319275,9606,101060226,NBPF19,-,-,MIM:614006|HGNC:HGNC:31999|Ensembl:ENSG0000027...,1,1q21.2,NBPF member 19,protein-coding,NBPF19,NBPF member 19,O,neuroblastoma breakpoint family member 19,20240407,-
1,26,NM_001278267,chr1,+,144146810,146467744,144158383,146466121,131,"144146810,144148789,144149726,144150981,144151...","144147021,144148892,144149941,144151054,144151...",0,NBPF20,cmpl,cmpl,"-1,-1,-1,-1,-1,-1,-1,-1,0,2,1,2,1,2,1,2,1,2,1,...",2307738,9606,100288142,NBPF20,-,-,MIM:614007|HGNC:HGNC:32000|Ensembl:ENSG0000016...,1,1q21.1,NBPF member 20,protein-coding,NBPF20,NBPF member 20,O,neuroblastoma breakpoint family member 20,20240403,-
2,12,NM_000109,chrX,-,31137340,33357505,31140035,33357382,79,"31137340,31144758,31152218,31164407,31165391,3...","31140047,31144790,31152311,31164531,31165635,3...",0,DMD,cmpl,cmpl,"0,1,1,0,2,2,2,2,2,0,2,0,1,2,1,1,2,1,0,0,1,0,2,...",2217347,9606,1756,DMD,-,BMD|CMD3B|DXS142|DXS164|DXS206|DXS230|DXS239|D...,MIM:300377|HGNC:HGNC:2928|Ensembl:ENSG00000198...,X,Xp21.2-p21.1,dystrophin,protein-coding,DMD,dystrophin,O,dystrophin|mutant dystrophin,20240506,-
3,2,NM_001351274,chr11,-,83166055,85339417,83170860,85337661,27,"83166055,83173044,83177750,83180243,83182668,8...","83170967,83173136,83177860,83180416,83182770,8...",0,DLG2,cmpl,cmpl,"1,2,0,1,1,1,0,2,2,1,0,2,2,2,1,0,1,2,0,0,0,0,0,...",2166801,9606,1740,DLG2,-,PPP1R58|PSD-93|PSD93|chapsyn-110,MIM:603583|HGNC:HGNC:2901|Ensembl:ENSG00000150...,11,11q14.1,discs large MAGUK scaffold protein 2,protein-coding,DLG2,discs large MAGUK scaffold protein 2,O,disks large homolog 2|channel-associated prote...,20240411,-
4,9,NM_033225,chr8,-,2792882,4852436,2796106,4851938,70,"2792882,2799993,2806820,2807752,2808635,281174...","2796266,2800126,2806908,2807865,2808797,281179...",0,CSMD1,cmpl,cmpl,"2,1,0,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...",2055832,9606,64478,CSMD1,-,PPP1R24,MIM:608397|HGNC:HGNC:14026|Ensembl:ENSG0000018...,8,8p23.2,CUB and Sushi multiple domains 1,protein-coding,CSMD1,CUB and Sushi multiple domains 1,O,CUB and sushi domain-containing protein 1|prot...,20240407,-


In [162]:
gene_fam = pd.read_csv(hgnc_gene_fam_file, sep=',')
gene_fam = gene_fam[gene_fam['typical_gene'].notna()].rename(columns={'id': 'fam'})
gene_fam_grp = gene_fam.groupby('typical_gene').agg({'fam': set})

In [163]:
gene = refgene_info.loc[
    refgene_info['type_of_gene'] == 'protein-coding',
    ['chrom', 'txStart', 'txEnd', 'GeneID', 'name2', 'name', 'strand']
].sort_values(['chrom', 'txStart', 'txEnd'])
gene.head()

,chrom,txStart,txEnd,GeneID,name2,name,strand
18195,chr1,69090,70008,79501,OR4F5,NM_001005484,+
18011,chr1,367658,368597,729759,OR4F29,NM_001005221,+
18008,chr1,621095,622034,81399,OR4F16,NM_001005277,-
9159,chr1,859302,879954,148398,SAMD11,NM_001385641,+
10490,chr1,879582,894636,26155,NOC2L,NM_015658,-


In [164]:
gene.rename(columns={
    'chrom': '#chrom', 'txStart': 'start', 'txEnd': 'end',
    'GeneID': 'gene_id', 'name2': 'symbol', 'name': 'transcript'
}).to_csv(gene_file, index=False, sep='\t')
print(gene_file)

/staging/software/lib/python3.9/site-packages/autocnv-1-py3.9.egg/data/gene.sorted.bed


In [165]:
!bgzip -cf {gene_file} > {gene_file}.gz
!tabix -fp bed {gene_file}.gz
!rm {gene_file}

In [166]:
# # prep an external omim gene list
# omim_gene_list_file = os.path.join(raw_data_dir, 'gene-list-key-lte3.xlsx')
# raw_omim_df = pd.read_excel(omim_gene_list_file)
## can not download gene-list-key-lt3.xlsx
## use https://www.omim.org/static/omim/data/mim2gene.txt to extract omim_gene instead
!wget -c -P {raw_data_dir} "https://www.omim.org/static/omim/data/mim2gene.txt"
omim_ori_file = os.path.join(raw_data_dir, 'mim2gene.txt')

--2024-05-11 14:00:49--  https://www.omim.org/static/omim/data/mim2gene.txt
Connecting to 192.168.134.12:1080... connected.
Proxy request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [167]:
raw_omim_df = pd.read_csv(omim_ori_file, sep='\t', skiprows=4)
print(raw_omim_df.columns)
raw_omim_df.head()

Index(['# MIM Number',
       'MIM Entry Type (see FAQ 1.3 at https://omim.org/help/faq)',
       'Entrez Gene ID (NCBI)', 'Approved Gene Symbol (HGNC)',
       'Ensembl Gene ID (Ensembl)'],
      dtype='object')


,# MIM Number,MIM Entry Type (see FAQ 1.3 at https://omim.org/help/faq),Entrez Gene ID (NCBI),Approved Gene Symbol (HGNC),Ensembl Gene ID (Ensembl)
0,100050,predominantly phenotypes,NaN,NaN,NaN
1,100070,phenotype,100329167.0,NaN,NaN
2,100100,phenotype,NaN,NaN,NaN
3,100200,predominantly phenotypes,NaN,NaN,NaN
4,100300,phenotype,NaN,NaN,NaN


In [168]:
omim_gene = set(raw_omim_df['Approved Gene Symbol (HGNC)'].dropna())
# with open(omim_gene_list_file) as f:
#     omim_gene = set(f.read().split('\n'))
omim_gene

{'TMEM160',
 'MSTO1',
 'H3-4',
 'MPZL3',
 'DDX42',
 'NPR2',
 'SMR3A',
 'SPINK2',
 'DDAH2',
 'CRIPTO',
 'MON1B',
 'SCGB2B2',
 'NOS2',
 'PRSS16',
 'MAST3',
 'SLC35E3',
 'ULBP2',
 'MAGI2',
 'LY75',
 'UGT1A9',
 'TENM4',
 'SCT',
 'ZNF703',
 'RNGTT',
 'COX6B1',
 'NBPF5P',
 'ARHGEF3',
 'CPPED1',
 'HMGCLL1',
 'PTGDR2',
 'NOD2',
 'RHOU',
 'PLA2G4B',
 'ZNF19',
 'B2M',
 'MCU',
 'DNA2',
 'LSM5',
 'MIR520F',
 'C2orf69',
 'DCAF1',
 'IRF6',
 'TNIP2',
 'MSH4',
 'FABP6',
 'PCDHB5',
 'EFEMP2',
 'C1QTNF3',
 'APOL3',
 'SFXN2',
 'BSG',
 'GOLPH3L',
 'SHISA9',
 'PARP9',
 'STOX1',
 'UQCR11',
 'TNFRSF6B',
 'FAM83G',
 'NRK',
 'SNORD114-1',
 'LTBP1',
 'INPP5A',
 'CD300H',
 'GMFB',
 'ITGAV',
 'HPX',
 'GATM',
 'RAB3C',
 'ZNF668',
 'CFAP44',
 'CRHBP',
 'SLC6A15',
 'ISG20',
 'MBOAT1',
 'HMGCS1',
 'SGK1',
 'CDC37',
 'FAAP100',
 'USP6',
 'ANKFY1',
 'SGMS1',
 'NFAM1',
 'SLC23A1',
 'SBF1',
 'EPM2AIP1',
 'BDNF',
 'AKT1',
 'ATP2A2',
 'GABRR2',
 'COP1',
 'RBM45',
 'PPP6R3',
 'TACC2',
 'UTP20',
 'TMEM175',
 'FBXL15',
 'CREL

In [169]:
omim_gene = refgene_info.loc[
    refgene_info['name2'].isin(omim_gene),
    ['chrom', 'txStart', 'txEnd', 'GeneID', 'name2', 'name', 'strand']
].sort_values(['chrom', 'txStart', 'txEnd'])
omim_gene.head()

,chrom,txStart,txEnd,GeneID,name2,name,strand
9159,chr1,859302,879954,148398,SAMD11,NM_001385641,+
10490,chr1,879582,894636,26155,NOC2L,NM_015658,-
14731,chr1,895963,901099,339451,KLHL17,NM_198317,+
14417,chr1,910577,917473,84808,PERM1,NM_001291367,-
18199,chr1,934343,935477,57801,HES4,NM_001142467,-


In [170]:
omim_gene.rename(columns={
    'chrom': '#chrom', 'txStart': 'start', 'txEnd': 'end',
    'GeneID': 'gene_id', 'name2': 'symbol', 'name': 'transcript'
}).to_csv(omim_gene_file, index=False, sep='\t')
print(omim_gene_file)

/staging/software/lib/python3.9/site-packages/autocnv-1-py3.9.egg/data/omim-gene.sorted.bed


In [171]:
!bgzip -cf {omim_gene_file} > {omim_gene_file}.gz
!tabix -fp bed {omim_gene_file}.gz
!rm {omim_gene_file}

In [172]:
curation_gene = pd.read_csv(clingen_gene_ori_file, sep='\t', dtype=str, skiprows=5)

In [173]:
hi_genes = set(
    curation_gene.loc[
        curation_gene['Haploinsufficiency Score'] == '3', '#Gene Symbol'
    ]
)
hi_gene = refgene_info.loc[
    refgene_info['name2'].isin(hi_genes),
    ['chrom', 'txStart', 'txEnd', 'GeneID', 'name2', 'name', 'strand']
].sort_values(['chrom', 'txStart', 'txEnd'])
hi_gene.head()

,chrom,txStart,txEnd,GeneID,name2,name,strand
51,chr1,6845513,7829766,23261,CAMTA1,NM_001349609,+
6453,chr1,17345220,17380527,6390,SDHB,NM_003000,-
3053,chr1,27022505,27108595,8289,ARID1A,NM_139135,+
14538,chr1,27860755,27930655,27245,AHDC1,NM_001371928,-
6966,chr1,43391025,43424539,6513,SLC2A1,NM_006516,-


In [174]:
hi_gene.rename(columns={
    'chrom': '#chrom', 'txStart': 'start', 'txEnd': 'end',
    'GeneID': 'gene_id', 'name2': 'symbol', 'name': 'transcript'
}).to_csv(hi_gene_file, index=False, sep='\t')

In [175]:
!bgzip -cf {hi_gene_file} > {hi_gene_file}.gz
!tabix -fp bed {hi_gene_file}.gz
!rm {hi_gene_file}

In [176]:
hi_cds = refgene_info.loc[
    (refgene_info['name2'].isin(hi_genes)) & (refgene_info['length'] != 0),
    ['chrom', 'cdsStart', 'cdsEnd', 'GeneID', 'name2', 'name']
].sort_values(['chrom', 'cdsStart', 'cdsEnd'])
hi_cds.head()

,chrom,cdsStart,cdsEnd,GeneID,name2,name
51,chr1,6845590,7826582,23261,CAMTA1,NM_001349609
6453,chr1,17345375,17380514,6390,SDHB,NM_003000
3053,chr1,27022894,27107247,8289,ARID1A,NM_139135
14538,chr1,27873814,27878626,27245,AHDC1,NM_001371928
6966,chr1,43392711,43424322,6513,SLC2A1,NM_006516


In [177]:
hi_cds.rename(columns={
    'chrom': '#chrom', 'cdsStart': 'start', 'cdsEnd': 'end',
    'GeneID': 'gene_id', 'name2': 'symbol', 'name': 'transcript'
}).to_csv(hi_cds_file, index=False, sep='\t')

In [178]:
!bgzip -cf {hi_cds_file} > {hi_cds_file}.gz
!tabix -fp bed {hi_cds_file}.gz
!rm {hi_cds_file}

In [179]:
hi_exons = refgene_info.loc[
    refgene_info['name2'].isin(hi_genes), ['chrom', 'exonStarts', 'exonEnds', 'GeneID', 'name2', 'name', 'strand']
].copy()
hi_exons.head()

,chrom,exonStarts,exonEnds,GeneID,name2,name,strand
2,chrX,"31137340,31144758,31152218,31164407,31165391,3...","31140047,31144790,31152311,31164531,31165635,3...",1756,DMD,NM_000109,-
26,chr7,"69063460,69364271,69583117,69599521,69900737,7...","69064948,69364484,69583219,69599557,69900767,7...",26053,AUTS2,NM_015570,+
31,chrX,"28605562,28807436,29301054,29414374,29417271,2...","28606164,28807542,29301334,29414561,29417425,2...",11141,IL1RAPL1,NM_014271,+
44,chr2,"50145642,50170841,50280408,50282092,50318460,5...","50149389,50170929,50280728,50282182,50318632,5...",9378,NRXN1,NM_001330084,-
51,chr1,"6845513,6880240,6885151,7151363,7309550,752788...","6845635,6880310,6885270,7151431,7309686,752796...",23261,CAMTA1,NM_001349609,+


In [180]:
hi_exons['exonStarts'] = hi_exons['exonStarts'].str.replace(r',$', '')
hi_exons['exonEnds'] = hi_exons['exonEnds'].str.replace(r',$', '')
hi_exons.head()

/tmp/ipykernel_136260/867614133.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  hi_exons['exonStarts'] = hi_exons['exonStarts'].str.replace(r',$', '')
/tmp/ipykernel_136260/867614133.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  hi_exons['exonEnds'] = hi_exons['exonEnds'].str.replace(r',$', '')


,chrom,exonStarts,exonEnds,GeneID,name2,name,strand
2,chrX,"31137340,31144758,31152218,31164407,31165391,3...","31140047,31144790,31152311,31164531,31165635,3...",1756,DMD,NM_000109,-
26,chr7,"69063460,69364271,69583117,69599521,69900737,7...","69064948,69364484,69583219,69599557,69900767,7...",26053,AUTS2,NM_015570,+
31,chrX,"28605562,28807436,29301054,29414374,29417271,2...","28606164,28807542,29301334,29414561,29417425,2...",11141,IL1RAPL1,NM_014271,+
44,chr2,"50145642,50170841,50280408,50282092,50318460,5...","50149389,50170929,50280728,50282182,50318632,5...",9378,NRXN1,NM_001330084,-
51,chr1,"6845513,6880240,6885151,7151363,7309550,752788...","6845635,6880310,6885270,7151431,7309686,752796...",23261,CAMTA1,NM_001349609,+


In [181]:
start = hi_exons['exonStarts'].str.split(',').apply(pd.Series).stack().reset_index()
start = start.rename(columns={'level_0': 'row', 0: 'start'})[['row', 'start']]
start['start'] = start['start'].astype(int)
end = hi_exons['exonEnds'].str.split(',').apply(pd.Series).stack().reset_index()
end = end.rename(columns={0: 'end'})['end'].astype(int)
position = start.join(end)
position.head()

,row,start,end
0,2,31137340,31140047
1,2,31144758,31144790
2,2,31152218,31152311
3,2,31164407,31164531
4,2,31165391,31165635


In [182]:
exon = position.merge(
    hi_exons[['chrom', 'GeneID', 'name2', 'name', 'strand']], how='left', left_on='row', right_index=True
)
exon = exon.sort_values(['chrom', 'start', 'end'])
exon.head()

,row,start,end,chrom,GeneID,name2,name,strand
128,51,6845513,6845635,chr1,23261,CAMTA1,NM_001349609,+
129,51,6880240,6880310,chr1,23261,CAMTA1,NM_001349609,+
130,51,6885151,6885270,chr1,23261,CAMTA1,NM_001349609,+
131,51,7151363,7151431,chr1,23261,CAMTA1,NM_001349609,+
132,51,7309550,7309686,chr1,23261,CAMTA1,NM_001349609,+


In [183]:
exon['+'] = exon.groupby(['name2', 'name'])['start'].rank('first', ascending=True).astype(int)
exon['-'] = exon.groupby(['name2', 'name'])['start'].rank('first', ascending=False).astype(int)
exon.head()
exon.tail()

,row,start,end,chrom,GeneID,name2,name,strand,+,-
1600,1189,154227753,154227875,chrX,2157,F8,NM_000132,-,25,2
1601,1189,154250684,154250998,chrX,2157,F8,NM_000132,-,26,1
5975,15336,154487519,154490514,chrX,116442,RAB39B,NM_171998,-,1,2
5976,15336,154493358,154493776,chrX,116442,RAB39B,NM_171998,-,2,1
6032,18569,2654895,2655723,chrY,6736,SRY,NM_003140,-,1,1


In [184]:
exon['exon'] = pd.concat([exon.loc[exon['strand'] == '+', '+'], exon.loc[exon['strand'] == '-', '-']])
exon.head()
exon.tail()

,row,start,end,chrom,GeneID,name2,name,strand,+,-,exon
1600,1189,154227753,154227875,chrX,2157,F8,NM_000132,-,25,2,2
1601,1189,154250684,154250998,chrX,2157,F8,NM_000132,-,26,1,1
5975,15336,154487519,154490514,chrX,116442,RAB39B,NM_171998,-,1,2,2
5976,15336,154493358,154493776,chrX,116442,RAB39B,NM_171998,-,2,1,1
6032,18569,2654895,2655723,chrY,6736,SRY,NM_003140,-,1,1,1


In [185]:
exon['last_exon'] = exon.groupby(['name2', 'name'])['exon'].transform('max') == exon['exon']
exon.head()
exon.tail()

,row,start,end,chrom,GeneID,name2,name,strand,+,-,exon,last_exon
1600,1189,154227753,154227875,chrX,2157,F8,NM_000132,-,25,2,2,False
1601,1189,154250684,154250998,chrX,2157,F8,NM_000132,-,26,1,1,False
5975,15336,154487519,154490514,chrX,116442,RAB39B,NM_171998,-,1,2,2,True
5976,15336,154493358,154493776,chrX,116442,RAB39B,NM_171998,-,2,1,1,False
6032,18569,2654895,2655723,chrY,6736,SRY,NM_003140,-,1,1,1,True


In [186]:
exon = exon[
    ['chrom', 'start', 'end', 'GeneID', 'name2', 'name', 'exon', 'last_exon']
].sort_values(['chrom', 'start', 'end'])
exon.head()

,chrom,start,end,GeneID,name2,name,exon,last_exon
128,chr1,6845513,6845635,23261,CAMTA1,NM_001349609,1,False
129,chr1,6880240,6880310,23261,CAMTA1,NM_001349609,2,False
130,chr1,6885151,6885270,23261,CAMTA1,NM_001349609,3,False
131,chr1,7151363,7151431,23261,CAMTA1,NM_001349609,4,False
132,chr1,7309550,7309686,23261,CAMTA1,NM_001349609,5,False


In [187]:
exon.rename(columns={
    'chrom': '#chrom',
    'GeneID': 'gene_id', 'name2': 'symbol', 'name': 'transcript'
}).to_csv(hi_exon_file, index=False, sep='\t')

In [188]:
!bgzip -cf {hi_exon_file} > {hi_exon_file}.gz
!tabix -fp bed {hi_exon_file}.gz
!rm {hi_exon_file}

In [189]:
last_exon = exon[exon['last_exon']]
last_exon.head()

,chrom,start,end,GeneID,name2,name,exon,last_exon
150,chr1,7826518,7829766,23261,CAMTA1,NM_001349609,23,True
4572,chr1,17345220,17345453,6390,SDHB,NM_003000,8,True
2930,chr1,27105513,27108595,8289,ARID1A,NM_139135,20,True
5919,chr1,27860755,27861427,27245,AHDC1,NM_001371928,9,True
4776,chr1,43391025,43392912,6513,SLC2A1,NM_006516,10,True


In [190]:
last_exon_region = last_exon['chrom'] + ':' + last_exon['start'].astype(str) + '-' + last_exon['end'].astype(str)
last_exon_region = last_exon_region.str.replace('chr', '')

In [191]:
last_exon_region.head()

150       1:7826518-7829766
4572    1:17345220-17345453
2930    1:27105513-27108595
5919    1:27860755-27861427
4776    1:43391025-43392912
dtype: object

In [192]:
need_fields = [
    'variants/CHROM', 'variants/POS', 'variants/REF', 'variants/ALT',
    'variants/AF_ESP', 'variants/AF_EXAC', 'variants/AF_TGP', 'variants/CLNSIG'
]

In [193]:
!tabix -f {clinvar_ori_vcf_file}

In [194]:
print(clinvar_file)
with open(clinvar_file, 'w') as f:
    headers = allel.read_vcf_headers(clinvar_ori_vcf_file)
    f.write(''.join(headers.headers))
    
    def fetch_variants(region):
        fields, samples, headers, it = allel.iter_vcf_chunks(
            clinvar_ori_vcf_file, fields=need_fields, alt_number=1, region=region
        )
        for variants, *_ in it:
            esp_filter = np.isnan(variants['variants/AF_ESP'])
            esp_filter[~esp_filter] |= variants['variants/AF_ESP'][~esp_filter] < 0.01

            exac_filter = np.isnan(variants['variants/AF_EXAC'])
            exac_filter[~exac_filter] |= variants['variants/AF_EXAC'][~exac_filter] < 0.01

            tgp_filter = np.isnan(variants['variants/AF_TGP'])
            tgp_filter[~tgp_filter] |= variants['variants/AF_TGP'][~tgp_filter] < 0.01

            pathogenic_filter = np.isin(
                variants['variants/CLNSIG'], ['Likely_pathogenic', 'Pathogenic', 'Pathogenic/Likely_pathogenic']
            )

            af_filter = esp_filter & exac_filter & tgp_filter & pathogenic_filter

            filtered_variants = {k: v[af_filter] for k, v in variants.items()}

            filtered_variants['variants/CHROM'] = 'chr' + filtered_variants['variants/CHROM']

            return allel.normalize_callset(filtered_variants)
    
    with Pool(processes=32) as pool:
        variants = pool.map(fetch_variants, last_exon_region)
    
    for names, callset in tqdm(filter(lambda x: x is not None, variants)):
        allel.write_vcf_data(f, names, callset, None, {field: np.nan for field in need_fields})

/staging/software/lib/python3.9/site-packages/autocnv-1-py3.9.egg/data/clinvar-pathogenic.sorted.vcf


360it [00:00, 2090.62it/s]


In [195]:
!sed -i 's/AF_.\+=nan;//' {clinvar_file}

In [196]:
!bgzip -cf {clinvar_file} > {clinvar_file}.gz
!tabix -fp vcf {clinvar_file}.gz
!rm {clinvar_file}

In [197]:
uhi_genes = set(
    curation_gene.loc[curation_gene['Haploinsufficiency Score'] == '40', '#Gene Symbol']
)

In [198]:
uhi_gene = refgene_info.loc[
    refgene_info['name2'].isin(uhi_genes),
    ['chrom', 'txStart', 'txEnd', 'GeneID', 'name2', 'name', 'strand']
].sort_values(['chrom', 'txStart', 'txEnd'])

In [199]:
uhi_gene.head()

,chrom,txStart,txEnd,GeneID,name2,name,strand
4986,chr1,32117864,32169618,1307,COL16A1,NM_001856,-
17425,chr1,40974390,40981722,64789,EXO5,NM_001346956,+
8295,chr1,55505220,55530525,255738,PCSK9,NM_174936,+
8352,chr1,171154438,171181824,2327,FMO2,NM_001460,+
9908,chr10,74870212,74891581,25961,NUDT13,NM_015901,+


In [200]:
uhi_gene.rename(columns={
    'chrom': '#chrom', 'txStart': 'start', 'txEnd': 'end',
    'GeneID': 'gene_id', 'name2': 'symbol', 'name': 'transcript'
}).to_csv(uhi_gene_file, index=False, sep='\t')

In [201]:
!bgzip -cf {uhi_gene_file} > {uhi_gene_file}.gz
!tabix -fp bed {uhi_gene_file}.gz
!rm {uhi_gene_file}

In [249]:
# ISCA-46748 invalid region moved to line 5 to skip
!sed -i '/^ISCA-46748/d' {clingen_region_ori_file}
region = pd.read_csv(clingen_region_ori_file, sep='\t', skiprows=5, dtype=str)
region.head()

,#ISCA ID,ISCA Region Name,cytoBand,Genomic Location,Haploinsufficiency Score,Haploinsufficiency Description,Haploinsufficiency PMID1,Haploinsufficiency PMID2,Haploinsufficiency PMID3,Haploinsufficiency PMID4,Haploinsufficiency PMID5,Haploinsufficiency PMID6,Triplosensitivity Score,Triplosensitivity Description,Triplosensitivity PMID1,Triplosensitivity PMID2,Triplosensitivity PMID3,Triplosensitivity PMID4,Triplosensitivity PMID5,Triplosensitivity PMID6,Date Last Evaluated,Haploinsufficiency Disease ID,Triplosensitivity Disease ID
0,ISCA-46747,"22q11.2 recurrent region (distal type III, E/F...",22q11.22q11.23,chr22:23119415-24994433,3,Sufficient evidence for dosage pathogenicity,NaN,NaN,NaN,NaN,NaN,NaN,1,Little evidence for dosage pathogenicity,19193630,22140377,19006218,26247050,NaN,NaN,2023-01-25,NaN,NaN
1,ISCA-46743,Xq25 region (includes STAG2),Xq25,chrX:123034319-123236519,3,Sufficient evidence for dosage pathogenicity,NaN,NaN,NaN,NaN,NaN,NaN,3,Sufficient evidence for dosage pathogenicity,23637084,24733578,26443594,25677961,31609727,NaN,2022-11-30,NaN,MONDO:0100038
2,ISCA-46742,7p22.1 region (includes ACTB),7p22.1,chr7:5536848-5799722,3,Sufficient evidence for dosage pathogenicity,27633570,29220674,32562408,NaN,NaN,NaN,1,Little evidence for dosage pathogenicity,26297194,21998864,22495914,25893121,25124455,27866048,2022-02-08,MONDO:0000508,MONDO:0100038
3,ISCA-46741,9p24.3 Region (includes DMRT1),9p24.3,chr9:1-1982069,2,Some evidence for dosage pathogenicity,20685758,22573722,23555275,32741963,NaN,NaN,Not yet evaluated,Not yet evaluated,NaN,NaN,NaN,NaN,NaN,NaN,2023-11-10,MONDO:0002145,NaN
4,ISCA-46739,Yq11.223 population region (DGV_Gold_Standard_...,Yq11.223,chrY:25853189-26146959,40,Dosage sensitivity unlikely,NaN,NaN,NaN,NaN,NaN,NaN,0,No evidence available,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-03,NaN,NaN


In [203]:
position = region['Genomic Location'].str.extract(r'(?P<chrom>chr\w+)\s*:\s*(?P<start>\d+)\s*-\s*(?P<end>\d+)')
position.head()

,chrom,start,end
0,chr22,23119415,24994433
1,chrX,123034319,123236519
2,chr7,5536848,5799722
3,chr9,1,1982069
4,chrY,25853189,26146959


In [204]:
position['start'] = position['start'].astype(int)
position['end'] = position['end'].astype(int)
position.head()

,chrom,start,end
0,chr22,23119415,24994433
1,chrX,123034319,123236519
2,chr7,5536848,5799722
3,chr9,1,1982069
4,chrY,25853189,26146959


In [205]:
region_pos = region.merge(position, how='left', left_index=True, right_index=True)
region_pos.head()

,#ISCA ID,ISCA Region Name,cytoBand,Genomic Location,Haploinsufficiency Score,Haploinsufficiency Description,Haploinsufficiency PMID1,Haploinsufficiency PMID2,Haploinsufficiency PMID3,Haploinsufficiency PMID4,Haploinsufficiency PMID5,Haploinsufficiency PMID6,Triplosensitivity Score,Triplosensitivity Description,Triplosensitivity PMID1,Triplosensitivity PMID2,Triplosensitivity PMID3,Triplosensitivity PMID4,Triplosensitivity PMID5,Triplosensitivity PMID6,Date Last Evaluated,Haploinsufficiency Disease ID,Triplosensitivity Disease ID,chrom,start,end
0,ISCA-46747,"22q11.2 recurrent region (distal type III, E/F...",22q11.22q11.23,chr22:23119415-24994433,3,Sufficient evidence for dosage pathogenicity,NaN,NaN,NaN,NaN,NaN,NaN,1,Little evidence for dosage pathogenicity,19193630,22140377,19006218,26247050,NaN,NaN,2023-01-25,NaN,NaN,chr22,23119415,24994433
1,ISCA-46743,Xq25 region (includes STAG2),Xq25,chrX:123034319-123236519,3,Sufficient evidence for dosage pathogenicity,NaN,NaN,NaN,NaN,NaN,NaN,3,Sufficient evidence for dosage pathogenicity,23637084,24733578,26443594,25677961,31609727,NaN,2022-11-30,NaN,MONDO:0100038,chrX,123034319,123236519
2,ISCA-46742,7p22.1 region (includes ACTB),7p22.1,chr7:5536848-5799722,3,Sufficient evidence for dosage pathogenicity,27633570,29220674,32562408,NaN,NaN,NaN,1,Little evidence for dosage pathogenicity,26297194,21998864,22495914,25893121,25124455,27866048,2022-02-08,MONDO:0000508,MONDO:0100038,chr7,5536848,5799722
3,ISCA-46741,9p24.3 Region (includes DMRT1),9p24.3,chr9:1-1982069,2,Some evidence for dosage pathogenicity,20685758,22573722,23555275,32741963,NaN,NaN,Not yet evaluated,Not yet evaluated,NaN,NaN,NaN,NaN,NaN,NaN,2023-11-10,MONDO:0002145,NaN,chr9,1,1982069
4,ISCA-46739,Yq11.223 population region (DGV_Gold_Standard_...,Yq11.223,chrY:25853189-26146959,40,Dosage sensitivity unlikely,NaN,NaN,NaN,NaN,NaN,NaN,0,No evidence available,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-03,NaN,NaN,chrY,25853189,26146959


In [206]:
func_region = region_pos.loc[
    (region_pos['Haploinsufficiency Score'].isin(['1', '2','3']))
     | (region_pos['Triplosensitivity Score'].isin(['1', '2','3'])),
     ['chrom', 'start', 'end', '#ISCA ID', 'ISCA Region Name']
].sort_values(['chrom', 'start', 'end'])
func_region.head()

,chrom,start,end,#ISCA ID,ISCA Region Name
484,chr1,834083,6289973,ISCA-37434,1p36 terminal region (includes GABRD)
490,chr1,145386507,145748064,ISCA-37428,1q21.1 recurrent (TAR syndrome) region (proxim...
494,chr1,146577486,147394506,ISCA-37421,"1q21.1 recurrent region (BP3-BP4, distal) (inc..."
459,chr1,243287730,245318287,ISCA-37493,1q43q44 terminal region (includes AKT3)
455,chr10,49389703,51053583,ISCA-37497,10q11.2 recurrent region (LCR C-LCR D)


In [207]:
func_region.rename(columns={
    'chrom': '#chrom',
    '#ISCA ID': 'isca_id', 'ISCA Region Name': 'name'
}).to_csv(func_region_file, sep='\t', index=False)

In [208]:
!bgzip -cf {func_region_file} > {func_region_file}.gz
!tabix -fp bed {func_region_file}.gz
!rm {func_region_file}

In [209]:
hi_region = region_pos.loc[
    region_pos['Haploinsufficiency Score'] == '3',
    ['chrom', 'start', 'end', '#ISCA ID', 'ISCA Region Name']
].sort_values(['chrom', 'start', 'end'])
hi_region.head()

,chrom,start,end,#ISCA ID,ISCA Region Name
484,chr1,834083,6289973,ISCA-37434,1p36 terminal region (includes GABRD)
494,chr1,146577486,147394506,ISCA-37421,"1q21.1 recurrent region (BP3-BP4, distal) (inc..."
459,chr1,243287730,245318287,ISCA-37493,1q43q44 terminal region (includes AKT3)
492,chr10,81682843,88739388,ISCA-37424,10q22.3q23.2 recurrent region (includes BMPR1A)
504,chr11,31803509,32510988,ISCA-37401,11p13 (WAGR syndrome) region


In [210]:
hi_region['omim_genes'] = hi_region.apply(
    lambda row: ','.join(omim_gene.loc[
        (omim_gene['chrom'] == row['chrom'])
        & (omim_gene['txEnd'] >= row['start'])
        & (omim_gene['txStart'] <= row['end']),
        'name2'
    ]),
    axis=1
)
hi_region.head()

,chrom,start,end,#ISCA ID,ISCA Region Name,omim_genes
484,chr1,834083,6289973,ISCA-37434,1p36 terminal region (includes GABRD),"SAMD11,NOC2L,KLHL17,PERM1,HES4,ISG15,AGRN,MIR2..."
494,chr1,146577486,147394506,ISCA-37421,"1q21.1 recurrent region (BP3-BP4, distal) (inc...","PRKAB2,FMO5,CHD1L,BCL9,ACP6,GJA5,GJA8"
459,chr1,243287730,245318287,ISCA-37493,1q43q44 terminal region (includes AKT3),"CEP170,SDCCAG8,AKT3,ZBTB18,ADSS2,CATSPERE,DESI..."
492,chr10,81682843,88739388,ISCA-37424,10q22.3q23.2 recurrent region (includes BMPR1A),"SFTPD,PLAC9,ANXA11,MAT1A,DYDC1,PRXL2A,NRG3,GHI..."
504,chr11,31803509,32510988,ISCA-37401,11p13 (WAGR syndrome) region,"ELP4,PAX6,RCN1,WT1,WT1-AS"


In [211]:
hi_region.rename(columns={
    'chrom': '#chrom',
    '#ISCA ID': 'isca_id', 'ISCA Region Name': 'name'
}).to_csv(hi_region_file, sep='\t', index=False)

In [212]:
!bgzip -cf {hi_region_file} > {hi_region_file}.gz
!tabix -fp bed {hi_region_file}.gz
!rm {hi_region_file}

In [213]:
uhi_region = region_pos.loc[
    region_pos['Haploinsufficiency Score'] == '40',
    ['chrom', 'start', 'end', '#ISCA ID', 'ISCA Region Name']
].sort_values(['chrom', 'start', 'end'])
uhi_region.head()

,chrom,start,end,#ISCA ID,ISCA Region Name
433,chr1,363999,401750,ISCA-46306,1p36.33 population region (gnomAD-SV_v2.1_DEL_...
432,chr1,1640999,1647000,ISCA-46307,1p36.33 population region (gnomAD-SV_v2.1_DEL_...
180,chr1,12886641,12942069,ISCA-46561,1p36.21 population region (DGV_Gold_Standard_J...
430,chr1,12896424,12930275,ISCA-46309,1p36.21 population region (gnomAD-SV_v2.1_DEL_...
179,chr1,12899395,12924283,ISCA-46562,1p36.21 population region (DGV_Gold_Standard_J...


In [214]:
uhi_region['genes'] = uhi_region.apply(
    lambda row: ','.join(gene.loc[
        (gene['chrom'] == row['chrom'])
        & (gene['txEnd'] >= row['start'])
        & (gene['txStart'] <= row['end']),
        'name2'
    ]),
    axis=1
)
uhi_region.head()

,chrom,start,end,#ISCA ID,ISCA Region Name,genes
433,chr1,363999,401750,ISCA-46306,1p36.33 population region (gnomAD-SV_v2.1_DEL_...,OR4F29
432,chr1,1640999,1647000,ISCA-46307,1p36.33 population region (gnomAD-SV_v2.1_DEL_...,"CDK11B,CDK11A"
180,chr1,12886641,12942069,ISCA-46561,1p36.21 population region (DGV_Gold_Standard_J...,"PRAMEF11,HNRNPCL1,PRAMEF2,PRAMEF4"
430,chr1,12896424,12930275,ISCA-46309,1p36.21 population region (gnomAD-SV_v2.1_DEL_...,"HNRNPCL1,PRAMEF2"
179,chr1,12899395,12924283,ISCA-46562,1p36.21 population region (DGV_Gold_Standard_J...,"HNRNPCL1,PRAMEF2"


In [215]:
uhi_region.rename(columns={
    'chrom': '#chrom',
    '#ISCA ID': 'isca_id', 'ISCA Region Name': 'name'
}).to_csv(uhi_region_file, sep='\t', index=False)

In [216]:
!bgzip -cf {uhi_region_file} > {uhi_region_file}.gz
!tabix -fp bed {uhi_region_file}.gz
!rm {uhi_region_file}

In [217]:
decipher = pd.read_csv(hi_pred_ori_file, sep='\t',skiprows=1, header=None, usecols=[3,])
decipher = decipher[3].str.split('|', expand=True).rename(columns={0: 'symbol', 1: 'hi_score', 2: 'hi_index'})
decipher['hi_index'] = decipher['hi_index'].str.replace('%', '').astype(float)
decipher = decipher.merge(gene, left_on='symbol', right_on='name2')
decipher.head()

,symbol,hi_score,hi_index,chrom,txStart,txEnd,GeneID,name2,name,strand
0,ANXA2R,0.000047349,100.00,chr5,43039472,43043349,389289,ANXA2R,NM_001382352,-
1,SCGB1D1,0.000054551,99.99,chr11,61957687,61961011,10648,SCGB1D1,NM_006552,+
2,IL31,0.000057228,99.99,chr12,122656575,122658768,386653,IL31,NM_001014336,-
3,BPIFA2,0.000061264,99.98,chr20,31749575,31769218,140683,BPIFA2,NM_001319164,+
4,MUCL1,0.000067779,99.98,chr12,55248299,55252171,118430,MUCL1,NM_058173,+


In [218]:
gnomad = pd.read_csv(gnomad_lof_ori_file, sep='\t', index_col=0, compression='gzip')
gnomad.head()

,transcript,obs_mis,exp_mis,oe_mis,mu_mis,possible_mis,obs_mis_pphen,exp_mis_pphen,oe_mis_pphen,possible_mis_pphen,obs_syn,exp_syn,oe_syn,mu_syn,possible_syn,obs_lof,mu_lof,possible_lof,exp_lof,pLI,pNull,pRec,oe_lof,oe_syn_lower,oe_syn_upper,oe_mis_lower,oe_mis_upper,oe_lof_lower,oe_lof_upper,constraint_flag,syn_z,mis_z,lof_z,oe_lof_upper_rank,oe_lof_upper_bin,oe_lof_upper_bin_6,n_sites,classic_caf,max_af,no_lofs,obs_het_lof,obs_hom_lof,defined,p,exp_hom_lof,classic_caf_afr,classic_caf_amr,classic_caf_asj,classic_caf_eas,classic_caf_fin,classic_caf_nfe,classic_caf_oth,classic_caf_sas,p_afr,p_amr,p_asj,p_eas,p_fin,p_nfe,p_oth,p_sas,transcript_type,gene_id,transcript_level,cds_length,num_coding_exons,gene_type,gene_length,exac_pLI,exac_obs_lof,exac_exp_lof,exac_oe_lof,brain_expression,chromosome,start_position,end_position
gene,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
MED13,ENST00000397786,871,1117.80,0.77921,0.000056,14195,314.0,529.75,0.59273,6708.0,422,387.53,1.0890,0.000019,4248,0.0,0.000005,1257.0,98.429,1.0,8.943600e-40,1.838300e-16,0.000000,1.005,1.180,0.736,0.824,0.000,0.030,NaN,-1.376500,2.6232,9.1935,0.0,0.0,0.0,2.0,0.000012,0.000008,124782.0,3.0,0.0,124785.0,0.000012,0.000018,0.000000,0.000000,0.000000,0.000000,0.000093,0.000009,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000093,0.000009,0.0,0.000000,protein_coding,ENSG00000108510,2,6522,30,protein_coding,122678,1.0,0.0,64.393,0.000000,NaN,17,60019966,60142643
NIPBL,ENST00000282516,846,1441.50,0.58688,0.000074,18540,158.0,543.10,0.29092,7135.0,496,495.01,1.0020,0.000025,5211,1.0,0.000009,1781.0,150.320,1.0,2.977300e-59,3.572400e-24,0.006653,0.930,1.079,0.554,0.621,0.001,0.032,NaN,-0.035119,5.5737,11.2860,1.0,0.0,0.0,2.0,0.000012,0.000008,125693.0,3.0,0.0,125696.0,0.000012,0.000018,0.000000,0.000000,0.000099,0.000000,0.000000,0.000000,0.0,0.000065,0.000000,0.000000,0.000099,0.000000,0.000000,0.000000,0.0,0.000065,protein_coding,ENSG00000164190,2,8412,46,protein_coding,189655,1.0,1.0,110.570,0.009044,NaN,5,36876861,37066515
SMC3,ENST00000361804,178,630.07,0.28251,0.000032,8109,21.0,182.52,0.11506,2197.0,215,203.25,1.0578,0.000010,2091,0.0,0.000005,937.0,79.490,1.0,2.785300e-32,2.191400e-13,0.000000,0.946,1.184,0.249,0.320,0.000,0.037,NaN,-0.647760,6.3999,8.2618,2.0,0.0,0.0,8.0,0.000032,0.000004,125731.0,8.0,0.0,125739.0,0.000032,0.000127,0.000000,0.000000,0.000099,0.000054,0.000000,0.000044,0.0,0.000033,0.000000,0.000000,0.000099,0.000054,0.000000,0.000044,0.0,0.000033,protein_coding,ENSG00000108055,2,3651,29,protein_coding,36946,1.0,0.0,58.523,0.000000,NaN,10,112327449,112364394
CNOT1,ENST00000317147,561,1295.90,0.43290,0.000069,15670,51.0,290.68,0.17545,3560.0,470,456.03,1.0306,0.000024,4564,1.0,0.000007,1440.0,125.030,1.0,2.992400e-49,4.562900e-20,0.007998,0.955,1.112,0.403,0.464,0.002,0.038,NaN,-0.514100,7.2546,10.2790,3.0,0.0,0.0,5.0,0.000020,0.000004,125740.0,4.0,0.0,125744.0,0.000016,0.000032,0.000000,0.000029,0.000000,0.000055,0.000000,0.000026,0.0,0.000000,0.000000,0.000029,0.000000,0.000054,0.000000,0.000018,0.0,0.000000,protein_coding,ENSG00000125107,2,7128,48,protein_coding,109936,1.0,3.0,90.130,0.033285,NaN,16,58553855,58663790
RLF,ENST00000372771,669,972.87,0.68766,0.000047,12682,107.0,321.14,0.33319,4151.0,358,352.62,1.0153,0.000017,3482,0.0,0.000004,1024.0,73.222,1.0,8.405500e-30,2.284200e-12,0.000000,0.930,1.108,0.645,0.733,0.000,0.040,NaN,-0.225180,3.4620,7.9294,4.0,0.0,0.0,1.0,0.000004,0.000004,125122.0,1.0,0.0,125123.0,0.000004,0.000002,0.000062,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000062,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,protein_coding,ENSG00000117000,2,5742,8,protein_coding,79549,1.0,0.0,43.607,0.000000,NaN,1,40627045,40706593


In [219]:
decipher = decipher.join(gnomad['pLI'], on='name2')
decipher = decipher.join(gnomad['oe_lof_upper'], on='name2')
decipher.head()

,symbol,hi_score,hi_index,chrom,txStart,txEnd,GeneID,name2,name,strand,pLI,oe_lof_upper
0,ANXA2R,0.000047349,100.00,chr5,43039472,43043349,389289,ANXA2R,NM_001382352,-,NaN,NaN
1,SCGB1D1,0.000054551,99.99,chr11,61957687,61961011,10648,SCGB1D1,NM_006552,+,1.294000e-07,1.968
2,IL31,0.000057228,99.99,chr12,122656575,122658768,386653,IL31,NM_001014336,-,1.144500e-02,1.715
3,BPIFA2,0.000061264,99.98,chr20,31749575,31769218,140683,BPIFA2,NM_001319164,+,8.012400e-07,1.495
4,MUCL1,0.000067779,99.98,chr12,55248299,55252171,118430,MUCL1,NM_058173,+,3.092300e-05,1.908


In [220]:
decipher = decipher.loc[
    (decipher['pLI'] >= 0.9) & (decipher['hi_index'] < 10) & (decipher['oe_lof_upper'] < 0.35),
    ['chrom', 'txStart', 'txEnd', 'GeneID', 'name2', 'name', 'pLI', 'hi_score']
].sort_values(['chrom', 'txStart', 'txEnd'])
decipher.head()

,chrom,txStart,txEnd,GeneID,name2,name,pLI,hi_score
17369,chr1,6845513,7829766,23261,CAMTA1,NM_001349609,1.00000,0.951338773
17427,chr1,8412463,8877699,473,RERE,NM_001042681,1.00000,0.962236687
17498,chr1,9751524,9788980,5293,PIK3CD,NM_001350235,0.99999,0.977549203
16243,chr1,10270763,10441661,23095,KIF1B,NM_015074,1.00000,0.70541308
17675,chr1,11166591,11322608,2475,MTOR,NM_004958,1.00000,0.997612705


In [221]:
decipher.rename(columns={
    'chrom': '#chrom', 'txStart': 'start', 'txEnd': 'end',
    'GeneID': 'gene_id', 'name2': 'symbol', 'name': 'transcript'
}).to_csv(decipher_gene_file, sep='\t', index=False)

In [222]:
!bgzip -cf {decipher_gene_file} > {decipher_gene_file}.gz
!tabix -fp bed {decipher_gene_file}.gz
!rm {decipher_gene_file}

In [223]:
ts_genes = set(
    curation_gene.loc[
        curation_gene['Triplosensitivity Score'] == '3', '#Gene Symbol'
    ]
)

In [224]:
ts_gene = refgene_info.loc[
    refgene_info['name2'].isin(ts_genes),
    ['chrom', 'txStart', 'txEnd', 'GeneID', 'name2', 'name', 'strand']
].sort_values(['chrom', 'txStart', 'txEnd'])
ts_gene.head()

,chrom,txStart,txEnd,GeneID,name2,name,strand
4327,chr5,126112827,126172712,4001,LMNB1,NM_005573,+
10786,chrX,103031780,103047547,5354,PLP1,NM_001305004,+


In [225]:
ts_gene.rename(columns={
    'chrom': '#chrom', 'txStart': 'start', 'txEnd': 'end',
    'GeneID': 'gene_id', 'name2': 'symbol', 'name': 'transcript'
}).to_csv(ts_gene_file, index=False, sep='\t')

In [226]:
!bgzip -cf {ts_gene_file} > {ts_gene_file}.gz
!tabix -fp bed {ts_gene_file}.gz
!rm {ts_gene_file}

In [227]:
uts_genes = set(
    curation_gene.loc[curation_gene['Triplosensitivity Score'] == '40', '#Gene Symbol']
)

In [228]:
uts_gene = refgene_info.loc[
    refgene_info['name2'].isin(uts_genes),
    ['chrom', 'txStart', 'txEnd', 'GeneID', 'name2', 'name', 'strand']
].sort_values(['chrom', 'txStart', 'txEnd'])
uts_gene.head()

,chrom,txStart,txEnd,GeneID,name2,name,strand
7271,chr13,103498381,103528345,2073,ERCC5,NM_000123,+
9351,chr19,45853094,45873831,2068,ERCC2,NM_000400,-
18393,chrX,6451658,6453159,51481,VCX3A,NM_016379,-


In [229]:
uts_gene.rename(columns={
    'chrom': '#chrom', 'txStart': 'start', 'txEnd': 'end',
    'GeneID': 'gene_id', 'name2': 'symbol', 'name': 'transcript'
}).to_csv(uts_gene_file, index=False, sep='\t')

In [230]:
!bgzip -cf {uts_gene_file} > {uts_gene_file}.gz
!tabix -fp bed {uts_gene_file}.gz
!rm {uts_gene_file}

In [231]:
ts_region = region_pos.loc[
    region_pos['Triplosensitivity Score'] == '3',
    ['chrom', 'start', 'end', '#ISCA ID', 'ISCA Region Name']
].sort_values(['chrom', 'start', 'end'])
ts_region.head()

,chrom,start,end,#ISCA ID,ISCA Region Name
494,chr1,146577486,147394506,ISCA-37421,"1q21.1 recurrent region (BP3-BP4, distal) (inc..."
503,chr15,22832519,28379874,ISCA-37404,"15q11.2q13 recurrent (PWS/AS) region (Class 1,..."
465,chr15,23747996,28379874,ISCA-37478,"15q11.2q13 recurrent (PWS/AS) region (Class 2,..."
505,chr16,29649997,30199852,ISCA-37400,"16p11.2 recurrent region (proximal, BP4-BP5) (..."
488,chr17,1247833,2588909,ISCA-37430,17p13.3 (Miller-Dieker syndrome) region (inclu...


In [232]:
ts_region['omim_genes'] = ts_region.apply(
    lambda row: ','.join(omim_gene.loc[
        (omim_gene['chrom'] == row['chrom'])
        & (omim_gene['txEnd'] >= row['start'])
        & (omim_gene['txStart'] <= row['end']),
        'name2'
    ]),
    axis=1
)
ts_region.head()

,chrom,start,end,#ISCA ID,ISCA Region Name,omim_genes
494,chr1,146577486,147394506,ISCA-37421,"1q21.1 recurrent region (BP3-BP4, distal) (inc...","PRKAB2,FMO5,CHD1L,BCL9,ACP6,GJA5,GJA8"
503,chr15,22832519,28379874,ISCA-37404,"15q11.2q13 recurrent (PWS/AS) region (Class 1,...","TUBGCP5,CYFIP1,NIPA2,NIPA1,MKRN3,MAGEL2,NDN,PW..."
465,chr15,23747996,28379874,ISCA-37478,"15q11.2q13 recurrent (PWS/AS) region (Class 2,...","MKRN3,MAGEL2,NDN,PWRN2,PWRN1,NPAP1,SNHG14,SNRP..."
505,chr16,29649997,30199852,ISCA-37400,"16p11.2 recurrent region (proximal, BP4-BP5) (...","SPN,QPRT,ZG16,KIF22,MAZ,PRRT2,PAGR1,MVP,CDIPT,..."
488,chr17,1247833,2588909,ISCA-37430,17p13.3 (Miller-Dieker syndrome) region (inclu...,"YWHAE,CRK,MYO1C,INPP5K,PITPNA,SLC43A2,SCARF1,R..."


In [233]:
ts_region.rename(columns={
    'chrom': '#chrom',
    '#ISCA ID': 'isca_id', 'ISCA Region Name': 'name'
}).to_csv(ts_region_file, sep='\t', index=False)

In [234]:
!bgzip -cf {ts_region_file} > {ts_region_file}.gz
!tabix -fp bed {ts_region_file}.gz
!rm {ts_region_file}

In [235]:
uts_region = region_pos.loc[
    region_pos['Triplosensitivity Score'] == '40',
    ['chrom', 'start', 'end', '#ISCA ID', 'ISCA Region Name']
].sort_values(['chrom', 'start', 'end'])
uts_region.head()

,chrom,start,end,#ISCA ID,ISCA Region Name
185,chr1,713039,756009,ISCA-46556,1p36.33 population region (DGV_Gold_Standard_J...
186,chr1,713039,756009,ISCA-46554,1p36.33 population region ((DGV_Gold_Standard_...
431,chr1,12894999,12940000,ISCA-46308,1p36.21 population region (gnomAD-SV_v2.1_DUP_...
184,chr1,12897609,12922238,ISCA-46557,1p36.21 population region (DGV_Gold_Standard_J...
418,chr1,104190999,104303200,ISCA-46321,1p21.1 population region (gnomAD-SV_v2.1_DUP_1...


In [236]:
uts_region['genes'] = uts_region.apply(
    lambda row: ','.join(gene.loc[
        (gene['chrom'] == row['chrom'])
        & (gene['txEnd'] >= row['start'])
        & (gene['txStart'] <= row['end']),
        'name2'
    ]),
    axis=1
)
uts_region.head()

,chrom,start,end,#ISCA ID,ISCA Region Name,genes
185,chr1,713039,756009,ISCA-46556,1p36.33 population region (DGV_Gold_Standard_J...,
186,chr1,713039,756009,ISCA-46554,1p36.33 population region ((DGV_Gold_Standard_...,
431,chr1,12894999,12940000,ISCA-46308,1p36.21 population region (gnomAD-SV_v2.1_DUP_...,"HNRNPCL1,PRAMEF2,PRAMEF4"
184,chr1,12897609,12922238,ISCA-46557,1p36.21 population region (DGV_Gold_Standard_J...,"HNRNPCL1,PRAMEF2"
418,chr1,104190999,104303200,ISCA-46321,1p21.1 population region (gnomAD-SV_v2.1_DUP_1...,"AMY1C,AMY1A,AMY1B"


In [237]:
uts_region.rename(columns={
    'chrom': '#chrom',
    '#ISCA ID': 'isca_id', 'ISCA Region Name': 'name'
}).to_csv(uts_region_file, sep='\t', index=False)

In [238]:
!bgzip -cf {uts_region_file} > {uts_region_file}.gz
!tabix -fp bed {uts_region_file}.gz
!rm {uts_region_file}

In [239]:
dgv = pd.read_csv(
    dgv_ori_file,
    sep='\t', names=['chrom', 'info'], usecols=[0, 8]
).drop_duplicates('info')
info = dgv['info'].str.extract(
    r'ID=(?P<id>[^;]+).*variant_sub_type=(?P<type>[^;]+).*inner_start=(?P<start>[^;]+).*inner_end=(?P<end>[^;]+).*Frequency=(?P<freq>\S+?)%;.*num_unique_samples_tested=(?P<sample>[^;]+)'
).astype({'start': int, 'end': int, 'sample': int, 'freq': float})
dgv = dgv.merge(info, left_index=True, right_index=True)
dgv['af'] = dgv['freq'] / 100
dgv = dgv[dgv['sample'] >= 1000].sort_values(['chrom', 'start', 'end'])
dgv.head()

,chrom,info,id,type,start,end,freq,sample,af
12,chr1,ID=gssvG9;Name=gssvG9;variant_type=CNV;variant...,gssvG9,Gain,49911,222421,1.13,1149,0.0113
54,chr1,ID=gssvG28;Name=gssvG28;variant_type=CNV;varia...,gssvG28,Gain,60905,97505,0.61,2609,0.0061
18651,chr1,ID=gssvL18;Name=gssvL18;variant_type=CNV;varia...,gssvL18,Loss,60905,97505,0.61,2609,0.0061
18627,chr1,ID=gssvL4;Name=gssvL4;variant_type=CNV;variant...,gssvL4,Loss,61724,346583,0.30,12364,0.0030
48,chr1,ID=gssvG24;Name=gssvG24;variant_type=CNV;varia...,gssvG24,Gain,125400,176984,0.73,1510,0.0073


In [240]:
def fetch_gene(gene, chrom, start, end):
    return ','.join(gene.loc[
        (gene['chrom'] == chrom) & (gene['txEnd'] >= start) & (gene['txStart'] <= end), 'name2'
    ])

In [241]:
with Pool(processes=7) as pool:
    dgv['genes'] = pool.starmap(fetch_gene, (
        (gene, row['chrom'], row['start'], row['end']) for _, row in dgv.iterrows()
    ), chunksize=70)
dgv.head()

,chrom,info,id,type,start,end,freq,sample,af,genes
12,chr1,ID=gssvG9;Name=gssvG9;variant_type=CNV;variant...,gssvG9,Gain,49911,222421,1.13,1149,0.0113,OR4F5
54,chr1,ID=gssvG28;Name=gssvG28;variant_type=CNV;varia...,gssvG28,Gain,60905,97505,0.61,2609,0.0061,OR4F5
18651,chr1,ID=gssvL18;Name=gssvL18;variant_type=CNV;varia...,gssvL18,Loss,60905,97505,0.61,2609,0.0061,OR4F5
18627,chr1,ID=gssvL4;Name=gssvL4;variant_type=CNV;variant...,gssvL4,Loss,61724,346583,0.30,12364,0.0030,OR4F5
48,chr1,ID=gssvG24;Name=gssvG24;variant_type=CNV;varia...,gssvG24,Gain,125400,176984,0.73,1510,0.0073,


In [242]:
dgv.loc[
    dgv['type'] == 'Gain', ['chrom', 'start', 'end', 'id', 'genes', 'af']
].rename(columns={'chrom': '#chrom'}).to_csv(dgv_gain_file, sep='\t', index=False)

In [243]:
!bgzip -cf {dgv_gain_file} > {dgv_gain_file}.gz
!tabix -fp bed {dgv_gain_file}.gz
!rm {dgv_gain_file}

In [244]:
dgv.loc[
    dgv['type'] == 'Loss', ['chrom', 'start', 'end', 'id', 'genes', 'af']
].rename(columns={'chrom': '#chrom'}).to_csv(dgv_loss_file, sep='\t', index=False)

In [245]:
!bgzip -cf {dgv_loss_file} > {dgv_loss_file}.gz
!tabix -fp bed {dgv_loss_file}.gz
!rm {dgv_loss_file}

In [246]:
gnomad = pd.read_csv(
    gnomad_control_ori_file, sep='\t',  dtype=str,
    usecols=[0, 1, 2, 3, 4, 37, 38, 73, 74, 107, 108, 141, 142, 175, 176, 241]
)
gnomad = gnomad[
    (gnomad['FILTER'] == 'PASS') & gnomad['svtype'].isin(['DEL', 'DUP'])
]
gnomad = gnomad[
    (gnomad['N_BI_GENOS'].astype(int) >= 1000) |
    (gnomad['AFR_N_BI_GENOS'].astype(int) >= 1000) |
    (gnomad['AMR_N_BI_GENOS'].astype(int) >= 1000) |
    (gnomad['EAS_N_BI_GENOS'].astype(int) >= 1000) |
    (gnomad['EUR_N_BI_GENOS'].astype(int) >= 1000)
]
gnomad['#chrom'] = 'chr' + gnomad['#chrom']
gnomad['start'] = gnomad['start'].astype(int)
gnomad['end'] = gnomad['end'].astype(int)
gnomad.head()

,#chrom,start,end,name,svtype,AF,N_BI_GENOS,AFR_AF,AFR_N_BI_GENOS,AMR_AF,AMR_N_BI_GENOS,EAS_AF,EAS_N_BI_GENOS,EUR_AF,EUR_N_BI_GENOS,FILTER
6,chr1,62398,62489,gnomAD-SV_v2.1_DEL_1_3,DEL,9.600000339560211e-05,5192,0.0,2014,0.0,609,0.0005840000230818987,856,0.0,1660,PASS
7,chr1,66349,66427,gnomAD-SV_v2.1_DEL_1_4,DEL,0.0004830000107176602,5177,0.0,2009,0.0024709999561309814,607,0.0,856,0.0006050000083632767,1652,PASS
15,chr1,136999,155500,gnomAD-SV_v2.1_DUP_1_3,DUP,0.138032004237175,5122,0.10211800038814545,1983,0.1157890036702156,570,0.3907710015773773,856,0.05963899940252304,1660,PASS
16,chr1,141474,155000,gnomAD-SV_v2.1_DUP_1_4,DUP,0.006382000166922808,4544,0.01448499970138073,1864,0.0,503,0.001905999961309135,787,0.00037200000951997936,1343,PASS
24,chr1,319999,327650,gnomAD-SV_v2.1_DEL_1_9,DEL,0.1377120018005371,5192,0.29294899106025696,2014,0.020524999126791954,609,0.08235999941825867,856,0.022590000182390213,1660,PASS


In [247]:
with Pool(processes=30) as pool:
    gnomad['genes'] = pool.starmap(fetch_gene, (
        (gene, row['#chrom'], row['start'], row['end']) for _, row in gnomad.iterrows()
    ), chunksize=70)

In [248]:
# gnomad['genes'] = pool.starmap(fetch_gene, (
#         (gene, row['#chrom'], row['start'], row['end']) for _, row in gnomad.iterrows()
#     ), chunksize=70)
gnomad['gene'] = gnomad.apply(lambda row: fetch_gene(gene, row['#chrom'], row['start'], row['end']), axis=1)
gnomad.head()

KeyboardInterrupt: 

In [ ]:
gnomad[
    gnomad['svtype'] == 'DEL'
][['#chrom', 'start', 'end', 'name', 'genes', 'AF', 'AFR_AF', 'AMR_AF', 'EAS_AF', 'EUR_AF']].rename(columns={
    'AF': 'af', 'AFR_AF': 'af_afr', 'AMR_AF': 'af_amr', 'EAS_AF': 'af_eas', 'EUR_AF': 'af_eur'
}).to_csv(gnomad_del_file, sep='\t', index=False)

In [ ]:
!bgzip -cf {gnomad_del_file} > {gnomad_del_file}.gz
!tabix -fp bed {gnomad_del_file}.gz
!rm {gnomad_del_file}

In [ ]:
gnomad.loc[
    gnomad['svtype'] == 'DUP',
    ['#chrom', 'start', 'end', 'name', 'genes', 'AF', 'AFR_AF', 'AMR_AF', 'EAS_AF', 'EUR_AF']
].rename(columns={
    'AF': 'af', 'AFR_AF': 'af_afr', 'AMR_AF': 'af_amr', 'EAS_AF': 'af_eas', 'EUR_AF': 'af_eur'
}).to_csv(gnomad_dup_file, sep='\t', index=False)

In [ ]:
!bgzip -cf {gnomad_dup_file} > {gnomad_dup_file}.gz
!tabix -fp bed {gnomad_dup_file}.gz
!rm {gnomad_dup_file}

In [256]:
refGene_df = read_gtf(exon_ori_file, result_type='pandas')
# refGene_df = refGene_df.applymap(lambda x: x.strip('"') if '"' in str(x) else x)
refGene_df.head()

INFO:root:Extracted GTF attributes: ['gene_id', 'transcript_id', 'gene_name', 'exon_number', 'exon_id']


,seqname,source,feature,start,end,score,strand,frame,gene_id,transcript_id,gene_name,exon_number,exon_id
0,chr1,refGene,transcript,11869,14362,NaN,+,0,LOC102725121,NR_148357,LOC102725121,,
1,chr1,refGene,exon,11869,12227,NaN,+,0,LOC102725121,NR_148357,LOC102725121,1,NR_148357.1
2,chr1,refGene,exon,12613,12721,NaN,+,0,LOC102725121,NR_148357,LOC102725121,2,NR_148357.2
3,chr1,refGene,exon,13221,14362,NaN,+,0,LOC102725121,NR_148357,LOC102725121,3,NR_148357.3
4,chr1,refGene,transcript,11874,14409,NaN,+,0,DDX11L1,NR_046018,DDX11L1,,


In [257]:
refGene_df = refGene_df[refGene_df['feature'] == 'exon']
refGene_df.head()

,seqname,source,feature,start,end,score,strand,frame,gene_id,transcript_id,gene_name,exon_number,exon_id
1,chr1,refGene,exon,11869,12227,NaN,+,0,LOC102725121,NR_148357,LOC102725121,1,NR_148357.1
2,chr1,refGene,exon,12613,12721,NaN,+,0,LOC102725121,NR_148357,LOC102725121,2,NR_148357.2
3,chr1,refGene,exon,13221,14362,NaN,+,0,LOC102725121,NR_148357,LOC102725121,3,NR_148357.3
5,chr1,refGene,exon,11874,12227,NaN,+,0,DDX11L1,NR_046018,DDX11L1,1,NR_046018.1
6,chr1,refGene,exon,12613,12721,NaN,+,0,DDX11L1,NR_046018,DDX11L1,2,NR_046018.2


In [272]:
gene_db = pd.read_csv(settings.GENE_DATABASE, sep='\t')
merge_df = gene_db.merge(refGene_df, left_on='transcript', right_on='transcript_id')[[
    '#chrom', 'start_y', 'end_y', 'gene_id_x', 'symbol', 'exon_number', 'transcript'
]]
merge_df.columns = [x.strip('_x').strip('_y') for x in merge_df.columns]
merge_df = merge_df.sort_values(['#chrom', 'start', 'end'])
merge_df.head()

,#chrom,start,end,gene_id,symbol,exon_number,transcript
0,chr1,69091,70008,79501,OR4F5,1,NM_001005484
1,chr1,367659,368597,729759,OR4F29,1,NM_001005221
2,chr1,367659,368597,81399,OR4F16,1,NM_001005277
3,chr1,879583,880180,26155,NOC2L,1,NM_015658
4,chr1,880437,880526,26155,NOC2L,2,NM_015658


In [273]:
merge_df.to_csv(settings.GENE_EXON_DATABASE.strip('.gz'), sep='\t', index=False)
print(settings.GENE_EXON_DATABASE.strip('.gz'), os.path.exists(settings.GENE_EXON_DATABASE.strip('.gz')))

/staging/software/lib/python3.9/site-packages/autocnv-1-py3.9.egg/data/exon.sorted.bed True


In [274]:
!bgzip -cf {settings.GENE_EXON_DATABASE.strip('.gz')} > {settings.GENE_EXON_DATABASE}

In [275]:
!tabix -fp bed {settings.GENE_EXON_DATABASE}
!rm {settings.GENE_EXON_DATABASE.strip('.gz')}

In [ ]:
# cnv_syndrome_loss cnv_syndrome_gain